In [2]:
print("Hello")

Hello


In [ ]:
#DAILY PERCENTAGE RETURNS DATA... need this to see how the price reacts to a given news release
# Remember, we are just doing the daily percentage gains, not going intraday yet, to keep things simple.


#Let's set up our Alpaca API...

# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

%matplotlib inline

# Load .env environment variables
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Verify that Alpaca key and secret were correctly loaded
print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpaca Secret Key type: {type(alpaca_secret_key)}")

# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

# Format current date as ISO format
today = pd.Timestamp("2020-07-14", tz="America/New_York").isoformat()

# Set the tickers
tickers = ["FB", "TWTR"]

# Set timeframe to one day ("1Day") for the Alpaca API
timeframe = "1Day"

# Get current closing prices for FB and TWTR
df_portfolio = alpaca.get_bars(
    tickers,
    timeframe,
    start = today,
    end = today
).df

# Display sample data
df_portfolio

# Format start and end dates as ISO format for one year period
start = pd.Timestamp("2019-07-14", tz="America/New_York").isoformat()
end = pd.Timestamp("2020-07-14", tz="America/New_York").isoformat()

# Get closing prices for FB and TWTR from the last year
df_portfolio_year = alpaca.get_bars(
    tickers,
    timeframe,
    start = start,
    end = end
).df

# Display sample data
df_portfolio_year.head(10)


In [ ]:

# Reorganize the DataFrame using the above data we just got from alpaca 

# Separate ticker data
FB = df_portfolio_year[df_portfolio_year['symbol']=='FB'].drop('symbol', axis=1)
TWTR = df_portfolio_year[df_portfolio_year['symbol']=='TWTR'].drop('symbol', axis=1)

# Concatenate the ticker DataFrames
df_portfolio_year = pd.concat([FB, TWTR],axis=1, keys=['FB','TWTR'])


In [ ]:
# GET GEOSPACIAL DATA
# We are interested in US companies that release 'hot' news.
# Where in the US are these companies located?
# We get their location, and we should be able to translate the "name" of that location into "coordinates"
# Once it is in "coordinates", we can plot that point on the Geo map using HVplot


# Frist step would be to get the locations of each company.

# Step 2 is to get access to a dataset that translates cities/towns in the US to coordinates.

# Lastly we can plot those coordinates on the map.

In [ ]:
# STEP 2: FIND DATASET TRANSLATING CITY NAME TO COORDINATES
# We can use this API: "GEOCODING API" https://openweathermap.org/api/geocoding-api 
# Here is a description from their website: 

# "Geocoding API is a simple tool that we have developed to ease the search for locations while working with geographic names and coordinates.
# Supporting API calls by geographical coordinates is the most accurate way to specify any location, that is why this method is integrated in all OpenWeather APIs. 
# However, this way is not always suitable for all users.
# Geocoding is the process of transformation of any location name into geographical coordinates, and the other way around (reverse geocoding).
# OpenWeather’s Geocoding API supports both the direct and reverse methods, working at the level of city names, areas and districts, countries and states:
# Direct geocoding converts the specified name of a location or zip/post code into the exact geographical coordinates;
# Reverse geocoding converts the geographical coordinates into the names of the nearby locations."

# We will use direct geocoding.

import requests
# import requests

# Create variable to hold request url
url = "http://api.openweathermap.org/geo/1.0/direct?q=Oakland&limit=5&appid=628878cc6edbe70ff1805be47e1af1ea"

# Add format specifier to request url
#url = url + "?format=json"

# Execute get request
requests.get(url)


# Execute GET request and store response
response_data = requests.get(url)
# q	required	City name, state code (only for the US) and country code divided by comma. Please use ISO 3166 country codes.
# appid	required	Your unique API key (you can always find it on your account page under the "API key" tab)
# limit	optional	Number of the locations in the API response (up to 5 results can be returned in the API response)

# http://api.openweathermap.org/geo/1.0/direct?q=London&limit=5&appid={API key}

#http://api.openweathermap.org/geo/1.0/direct?q=Oakland&limit=5&appid={3fcc50c01f481adc6307ae95cede2362}

In [ ]:
print(response_data.content) #I think I am getting an error because I need to confirm my email. However, that email is taking a while to arrive...